In [198]:
from synthetic_data import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from get_real_data import *

from preprocessing import preprocess
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import MaxPooling2D, LSTM, Dense, Masking, Conv1D, Conv2D, Flatten, MaxPooling1D, Input, Concatenate, Dropout, AveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop


In [2]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise=False)
X1a, y1a, X2a, y2a, X3a, y3a, X4a, y4a = augmentation(X1, y1, X2, y2, X3, y3, X4, y4, noise=False)

X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=False, general=False)
X_ris_wedga, y_ris_wedga, X_fal_wedga, y_fal_wedga, X_d_topa, y_d_topa, X_d_bottoma, y_d_bottoma = augmentation(X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom, noise=False)

X_ris_wedg = X1a + X_ris_wedga
y_ris_wedg = y1a + y_ris_wedga

X_train_ris, y_train_p_ris, y_train_dates_ris, X_test_ris, y_test_p_ris, y_test_dates_ris = preprocess(X_ris_wedg, y_ris_wedg)

X_fal_wedg = X2a + X_fal_wedga
y_fal_wedg = y2a + y_fal_wedga

X_train_fal, y_train_p_fal, y_train_dates_fal, X_test_fal, y_test_p_fal, y_test_dates_fal = preprocess(X_fal_wedg, y_fal_wedg)

X_d_top = X3a + X_d_topa
y_d_top = y3a + y_d_topa

X_train_dt, y_train_p_dt, y_train_dates_dt, X_test_dt, y_test_p_dt, y_test_dates_dt = preprocess(X_d_top, y_d_top)

X_d_bottom = X4a + X_d_bottoma
y_d_bottom = y4a + y_d_bottoma

X_train_db, y_train_p_db, y_train_dates_db, X_test_db, y_test_p_db, y_test_dates_db = preprocess(X_d_bottom, y_d_bottom)

X_full = X_ris_wedg + X_fal_wedg + X_d_top + X_d_bottom
y_full = y_ris_wedg + y_fal_wedg + y_d_top + y_d_bottom
X_train_full, y_train_p_full, y_train_dates_full, X_test_full, y_test_p_full, y_test_dates_full = preprocess(X_full, y_full)


2023-09-05 15:22:31.397350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-05 15:22:31.399624: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [133]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
X_train_preprocessed, y_train_p, y_train_dates, X_test_preprocessed, y_test_p, y_test_dates = get_data()

In [126]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise=False)
X1a, y1a, X2a, y2a, X3a, y3a, X4a, y4a = augmentation(X1, y1, X2, y2, X3, y3, X4, y4, noise=False)


In [127]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime
import random



i = random.randint(0,100)
print(i)

def plot_candles(X, y):
    df = pd.DataFrame(X, columns=['open', 'high', 'low', 'close'])
    df = df[(df != -100).all(axis=1)]

    fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df.open,
                    high=df.high,
                    low=df.low,
                    close=df.close)])
        
    y_min, y_max, *_ = y
    fig.add_vline(x=y_min, line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x=y_max, line_width=3, line_dash="dash", line_color="green")
    
    fig.show()

43


In [128]:
for i in range(10, 15):
    X = X2a[i]
    y = y2a[i]
    plot_candles(X, y)

* higher level == Asses general trend 
* intermediate levels == Asses specific trend 
* lower level == Asses dates 

* descending triangle, falling wedge, downtrend, descending channel 
* ascending triangle, rising wedge, uptrend, vertical run up
* double bottom, hs bottom, support
* double top, hs top, resistance

* Tryout without synthetic data, improve the data if needed

In [ ]:
X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=False, general=False)
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise=False)
X = X1 + X2 + X3 + X4 + X_ris_wedg, X_fal_wedg, X_d_top, X_d_bottom
y = y1 + y2 + y3 + y4 + y_ris_wedg, y_fal_wedg, y_d_top, y_d_bottom
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train, y_train = augmentate(X_train, y_train, noise=True)
X_test, y_test = augmentate(X_test, y_test, noise=True)
X_train_preprocessed, y_train_p, y_train_dates = preprocess(X_train, y_train)
X_test_preprocessed, y_test_p, y_test_dates = preprocess(X_test, y_test)



In [ ]:
X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=False, general=True)

In [ ]:
X_rw_noise, y_rw_noise, X_fw_noise, y_fw_noise, X_dt_noise, y_dt_noise, X_db_noise, y_db_noise = get_hack_noise(X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom, synth=True, real=True, f=True)

In [ ]:
X_synth = X_ris_wedg + X_rw_noise + X_fal_wedg + X_fw_noise + X_d_top + X_dt_noise + X_d_bottom + X_db_noise
y_synth = y_ris_wedg + y_rw_noise + y_fal_wedg + y_fw_noise + y_d_top + y_dt_noise + y_d_bottom + y_db_noise

In [ ]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise=False)

In [ ]:
X1_noise, y1_noise, X2_noise, y2_noise, X3_noise, y3_noise, X4_noise, y4_noise = get_hack_noise(X1, y1, X2, y2, X3, y3, X4, y4, synth=False, real=True, f=False)

In [ ]:
X_up, y_up, X_down, y_down = get_up_down(patterns=["uptrend","downtrend"])

In [ ]:
X_real = X1 + X1_noise + X2 + X2_noise + X3 + X3_noise + X4 + X4_noise + X_up + X_down
y_real = y1 + y1_noise + y2 + y2_noise + y3 + y3_noise + y4 + y4_noise + y_up + y_down


In [ ]:
X = X_real + X_synth
y = y_real + y_synth

In [ ]:
X_train_preprocessed, y_train_p, y_train_dates, X_test_preprocessed, y_test_p, y_test_dates = preprocess(X_synth, y_synth)

* get real data
* data separation in 3 windows 2 with no pattern 
    the one with pattern give it slack in both sides and make it 10 times with that slack
* flip the data upside down and give it the opposite name

X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
X4 = np.array(X4)
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)
y4 = np.array(y4)


In [4]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise=False)


In [5]:
X1a, y1a, X2a, y2a, X3a, y3a, X4a, y4a = augmentation(X1, y1, X2, y2, X3, y3, X4, y4, noise=True)

In [6]:
Xa = X1a + X2a +X3a + X4a
ya = y1a + y2a +y3a + y4a

In [7]:
X_train_a, y_train_p_a, y_train_dates_a, X_test_a, y_test_p_a, y_test_dates_a = preprocess(Xa, ya)

In [8]:
X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=False, general=False)

In [9]:
X_ris_wedga, y_ris_wedga, X_fal_wedga, y_fal_wedga, X_d_topa, y_d_topa, X_d_bottoma, y_d_bottoma = augmentation(X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom, noise=True)

In [11]:
X_syntha = X_ris_wedga + X_fal_wedga + X_d_topa + X_d_bottoma
y_syntha = y_ris_wedga + y_fal_wedga + y_d_topa + y_d_bottoma

In [12]:
X_c = Xa + X_syntha
y_c = ya + y_syntha

In [13]:
X_train_c, y_train_p_c, y_train_dates_c, X_test_c, y_test_p_c, y_test_dates_c = preprocess(X_c, y_c)

In [ ]:
x_cp=[]
y_cp=[]
y_cd=[]

for i in range(len(X_train_c)):
    if list(y_train_dates_c[i]) != [-1,-1]:
        x_cp.append(X_train_c[i])
        y_cd.append(y_train_dates_c[i])
        y_cp.append(y_train_p_c[i])

y_cp = tf.convert_to_tensor(y_cp, np.int16)
x_cp = tf.convert_to_tensor(x_cp, np.float32)
y_cd = tf.convert_to_tensor(y_cd, np.int16)


### We have to change with the functional api, because is a two problem one for regresion and other for classification

### We have to add an intermediate layer

### Look into generative models

In [ ]:
def initialize_model_CNN():
       
    # Input layer
    input_shape = X_train_preprocessed.shape[1:]
    input_layer = Input(shape=input_shape)

    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    conv1d_1 = Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(masked_input)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(32, activation='relu', kernel_size=3)(maxpool_1)
    flatten = Flatten()(conv1d_2)
    dense_1 = Dense(units=200, activation='relu')(flatten)
    dense_2 = Dense(units=100, activation='relu')(dense_1)
    dense_3 = Dense(units=16, activation='relu')(dense_2)
    dense_4 = Dense(units=16, activation='relu')(dense_3)

    # Classification output
    classification_output = Dense(units=4, activation='softmax', name='classification_output')(dense_4)

    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(dense_4)

    # Create the model
    model = Model(inputs=input_layer, outputs=[classification_output, regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='adam',
                loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
                metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})


    
    return model

In [ ]:

es = EarlyStopping(patience = 10, restore_best_weights=True)

model = initialize_model_CNN()

history = model.fit(
    x=X_train_preprocessed,
    y={'classification_output': y_train_p, 'regression_output': y_train_dates},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
evaluation = model.evaluate(
    x=X_test_preprocessed,  # Test input data
    y={'classification_output': y_test_p, 'regression_output': y_test_dates},  # Dictionary of test labels
    verbose=1  # Set to 1 for progress updates during evaluation
)
evaluation

In [ ]:
def plot_clas_reg(val_clas="val_classification_output_accuracy", val_mae="val_regression_output_mae"):
    clas = history.history[val_clas]
    reg = history.history[val_mae]
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('Horizontally stacked subplots')
    ax1.plot(clas)
    ax2.plot(reg)

In [ ]:
plot_clas_reg()

#### Model with the classification target as an intermediate layer

In [ ]:
def two_layer_model():
    
    # Input layer
    input_shape = X_train_preprocessed.shape[1:]
    input_layer = Input(shape=input_shape)

    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    conv1d_1 = Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(masked_input)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(32, activation='relu', kernel_size=3)(maxpool_1)
    flatten = Flatten()(conv1d_2)
    dense_1 = Dense(units=200, activation='relu')(flatten)
    dense_2 = Dense(units=100, activation='relu')(dense_1)
    dense_3 = Dense(units=16, activation='relu')(dense_2)
    dense_4 = Dense(units=16, activation='relu')(dense_3)

    # Classification output
    classification_output = Dense(units=5, activation='softmax', name='classification_output')(dense_4)

    
    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(dense_4)

    # Create the combined model with classification as an intermediate output
    combined_model = Model(inputs=input_layer, outputs=[classification_output, regression_output])

    # Compile the combined model
    combined_model.compile(optimizer='adam',
                        loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
                        metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})

    return combined_model


In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)

model_1 = two_layer_model()

history = model_1.fit(
    x=X_train_preprocessed,
    y={'classification_output': y_train_p, 'regression_output': y_train_dates},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
evaluation_1 = model_1.evaluate(
    x=X_test_preprocessed,  # Test input data
    y={'classification_output': y_test_p, 'regression_output': y_test_dates},  # Dictionary of test labels
    verbose=1  # Set to 1 for progress updates during evaluation
)
evaluation_1

#### Conditional intermediate layer

In [ ]:
def conditional_regression_model():
    # Input layer
    # Input layer
    input_shape = X_train_preprocessed.shape[1:]
    input_layer = Input(shape=input_shape)


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    conv1d_1 = Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(masked_input)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(32, activation='relu', kernel_size=3)(maxpool_1)
    flatten = Flatten()(conv1d_2)
    dense_1 = Dense(units=200, activation='relu')(flatten)
    dropout_1 = Dropout(rate=0.5)(dense_1)
    dense_2 = Dense(units=128, activation='relu')(dropout_1)
    dropout_2 = Dropout(rate=0.5)(dense_2)
    dense_3 = Dense(units=64, activation='relu')(dropout_2)
    dropout_3 = Dropout(rate=0.5)(dense_3)
    dense_4 = Dense(units=32, activation='relu')(dense_3)

    # Classification output
    classification_output = Dense(units=4, activation='softmax', name='classification_output')(dense_4)

    # Combine input and classification output for regression
    regression_input = Concatenate()([dense_4, classification_output])

    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(regression_input)

    # Create the combined model
    model = Model(inputs=input_layer, outputs=[classification_output, regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='adam',
                  loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
                  metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})

    return model


In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)

model_2 = conditional_regression_model()

history = model_2.fit(
    x=X_train_preprocessed,
    y={'classification_output': y_train_p, 'regression_output': y_train_dates},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
evaluation_2 = model_2.evaluate(
    x=X_test_preprocessed,  # Test input data
    y={'classification_output': y_test_p, 'regression_output': y_test_dates},  # Dictionary of test labels
    verbose=1  # Set to 1 for progress updates during evaluation
)


In [ ]:
plot_clas_reg()

#### Finding better parameters for the neural network

In [ ]:
def conditional_regression_model_1():
    # Input layer
    # Input layer
    input_shape = X_train_preprocessed.shape[1:]
    input_layer = Input(shape=input_shape)


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    conv1d_1 = Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(masked_input)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(32, activation='relu', kernel_size=3)(maxpool_1)
    
    flatten = Flatten()(conv1d_2)
    dense_1 = Dense(units=192, activation='relu')(flatten)
    dropout_1 = Dropout(rate=0.5)(dense_1)
    dense_2 = Dense(units=128, activation='relu')(dropout_1)
    dropout_2 = Dropout(rate=0.5)(dense_2)
    dense_3 = Dense(units=64, activation='relu')(dropout_2)
    dropout_3 = Dropout(rate=0.5)(dense_3)
    dense_4 = Dense(units=32, activation='relu')(dropout_3)
    

    # Classification output
    classification_output = Dense(units=4, activation='softmax', name='classification_output')(dense_4)

    # Combine input and classification output for regression
    regression_input = Concatenate()([dense_4, classification_output])

    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(regression_input)

    # Create the combined model
    model = Model(inputs=input_layer, outputs=[classification_output, regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='adam',
                  loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
                  metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})

    return model


In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_3 = conditional_regression_model_1()

history = model_3.fit(
    x=X_train_preprocessed,
    y={'classification_output': y_train_p, 'regression_output': y_train_dates},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
!pip freeze | grep pydot

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_3)

In [ ]:
plt.plot(history.history["loss"])

In [ ]:
evaluation_3 = model_3.evaluate(
    x=X_test_preprocessed,  # Test input data
    y={'classification_output': y_test_p, 'regression_output': y_test_dates},  # Dictionary of test labels
    verbose=1  # Set to 1 for progress updates during evaluation
)


In [ ]:
X_train_synth, y_train_p_synth, y_train_dates_synth, X_test_synth, y_test_p_synth, y_test_dates_synth = preprocess(X_synth, y_synth)

In [ ]:
def conditional_regression_model_2():
    # Input layer
    # Input layer
    input_shape = X_train_c.shape[1:]
    input_layer = Input(shape=input_shape)


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    #conv1d_1 = Conv1D(8, activation='relu', kernel_size=3)(masked_input)
    #maxpool_1 = AveragePooling1D(pool_size=2)(conv1d_1)
    #conv1d_2 = Conv1D(16, activation='relu', kernel_size=5)(maxpool_1)
    #maxpool_2 = AveragePooling1D(pool_size=2)(conv1d_2)
    #conv1d_3 = Conv1D(32, activation='relu', kernel_size=5)(maxpool_2)
    #maxpool_3 = AveragePooling1D(pool_size=2)(conv1d_3)
    #conv1d_4 = Conv1D(64, activation='relu', kernel_size=5)(maxpool_3)
    
    lstm_1 = LSTM(units=16, return_sequences=True)(masked_input)
    # lstm_2 = LSTM(units=8, return_sequences=True)(lstm_1)


    # Add the second LSTM layer
    lstm_3 = LSTM(units=16, return_sequences=False)(lstm_1)

    
    
    flatten = Flatten()(lstm_3)
    dense_1 = Dense(units=32, activation='relu')(flatten)
    #dropout_1 = Dropout(rate=0.5)(dense_1)
    # dense_2 = Dense(units=32, activation='relu')(dense_1)
    
    

    # Classification output
    classification_output = Dense(units=5, activation='softmax', name='classification_output')(dense_1)

    

    # Create the combined model

    
    # Create the combined model
    model = Model(inputs=input_layer, outputs=[classification_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='rmsprop',
                  loss={'classification_output': 'categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy'})

    return model


In [ ]:
_,counts = np.unique(y_train_p_c, axis=0, return_counts=True)
max(counts)/sum(counts)

In [ ]:
a = tf.keras.models.load_model("model_patterns_a")


In [ ]:
a.layers

In [ ]:
model_4.save("model_patterns_a")

In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_4 = conditional_regression_model_2()

history = model_4.fit(
    x=X_train_c,
    y={'classification_output': y_train_p_c},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
def conditional_regression_model_4():
    # Input layer
    # Input layer
    input_shape = X_train_a.shape[1:]
    input_layer = Input(shape=input_shape)


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    #conv1d_1 = Conv1D(8, activation='relu', kernel_size=3)(masked_input)
    #maxpool_1 = AveragePooling1D(pool_size=2)(conv1d_1)
    #conv1d_2 = Conv1D(16, activation='relu', kernel_size=5)(maxpool_1)
    #maxpool_2 = AveragePooling1D(pool_size=2)(conv1d_2)
    #conv1d_3 = Conv1D(32, activation='relu', kernel_size=5)(maxpool_2)
    #maxpool_3 = AveragePooling1D(pool_size=2)(conv1d_3)
    #conv1d_4 = Conv1D(64, activation='relu', kernel_size=5)(maxpool_3)
    
    lstm_1 = LSTM(units=16, return_sequences=True)(masked_input)
    # lstm_2 = LSTM(units=8, return_sequences=True)(lstm_1)


    # Add the second LSTM layer
    lstm_3 = LSTM(units=16, return_sequences=False)(lstm_1)

    
    
    flatten = Flatten()(lstm_3)
    dense_1 = Dense(units=32, activation='relu')(flatten)
    #dropout_1 = Dropout(rate=0.5)(dense_1)
    # dense_2 = Dense(units=32, activation='relu')(dense_1)
    
    

    # Classification output
    classification_output = Dense(units=5, activation='softmax', name='classification_output')(dense_1)

    

    # Create the combined model

    
    # Create the combined model
    model = Model(inputs=input_layer, outputs=[classification_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='rmsprop',
                  loss={'classification_output': 'categorical_crossentropy'},
                  metrics={'classification_output': 'accuracy'})

    return model


In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_5 = conditional_regression_model_4()

history = model_5.fit(
    x=X_train_a,
    y={'classification_output': y_train_p_a},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
def conditional_regression_model_3():
    # Input layer
    # Input layer
    

    pretrained_model = tf.keras.models.load_model("model_patterns_c")
    for layer in pretrained_model.layers:
        layer.trainable = False
    
    input_shape = x_cp.shape[1:]
    input_layer = Input(shape=input_shape)
        
    # model = tf.keras.Sequential()

    # # Add the pretrained model as the first layer (non-trainable)
    # model.add(pretrained_model.layers[2])

    # # Create your custom layers using model.add()
    # model.add(Conv1D(32, kernel_size=5, activation='relu'))
    # model.add(AveragePooling1D(pool_size=2))
    # model.add(Conv1D(32, kernel_size=3, activation='relu'))
    # model.add(Flatten())
    # model.add(Dense(32, activation='relu'))
    # model.add(Dense(2, activation='linear'))  # Output layer for regression


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    loaded_model = tf.keras.models.load_model("model_patterns_c")(masked_input)
    loaded_model.trainable = False
    lstms_1 = pretrained_model.layers[2](masked_input)
    # conv1d_1 = Conv1D(8, activation='relu', kernel_size=3)(masked_input)
    # maxpool_1 = AveragePooling1D(pool_size=2)(conv1d_1)
    # conv1d_2 = Conv1D(16, activation='relu', kernel_size=5)(maxpool_1)
    # #maxpool_2 = AveragePooling1D(pool_size=2)(conv1d_2)
    #conv1d_3 = Conv1D(32, activation='relu', kernel_size=5)(maxpool_2)
    #maxpool_3 = AveragePooling1D(pool_size=2)(conv1d_3)
    #conv1d_4 = Conv1D(64, activation='relu', kernel_size=5)(maxpool_3)
    
    # lstm_1 = LSTM(units=16, return_sequences=True)(conv1d_2)
    #lstm_2 = LSTM(units=16, return_sequences=False)(lstm_1)
    pretraining = pretrained_model(input_layer)

    # Add the second LSTM layer
    lstm_3 = LSTM(units=16, return_sequences=False)(lstm_1)

    flatten = Flatten()(pretr)
    #concat = Concatenate()([pretraining, flatten])
    dense_1 = Dense(units=32, activation='relu')(flatten)
    dropout_1 = Dropout(rate=0.5)(dense_1)
    dense_2 = Dense(units=32, activation='relu')(dropout_1)
    

    # Regression output
    regression_output = Dense(units=443, activation='linear', name='regression_output')(dense_2)

    # Create the combined model
    model = Model(inputs=[input_layer], outputs=[regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=0.001), metrics=['mae'])

    return model

    


In [ ]:
es = EarlyStopping(patience = 10, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_5 = conditional_regression_model_3()


history = model_5.fit(
    x=x_cp,
    y={"regression_output" : y_cp},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [ ]:
X_train_preprocessed, y_train_p, y_train_dates

In [148]:
def ini_model():
    input_layer = Input(shape=X_train_preprocessed.shape[1:])
    masked_input = Masking(mask_value=-1)(input_layer)

 
    lstm_1 = LSTM(units=16, return_sequences=True)(masked_input)

    lstm_3 = LSTM(units=16, return_sequences=False)(lstm_1)

    
    
    flatten = Flatten()(lstm_3)
    dense_1 = Dense(units=32, activation='relu')(flatten)
    
    output_layer = Dense(5, activation='softmax')(dense_1)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [149]:
es = EarlyStopping(patience = 3, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model = ini_model()


history = model.fit(
    x=X_train_preprocessed,
    y= y_train_p,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
931/931 [==============================] - 190s 201ms/step - loss: 1.2602 - accuracy: 0.5208 - val_loss: 1.1169 - val_accuracy: 0.6017
Epoch 2/50
931/931 [==============================] - 187s 200ms/step - loss: 1.1659 - accuracy: 0.5524 - val_loss: 1.0900 - val_accuracy: 0.6078
Epoch 3/50
931/931 [==============================] - ETA: 0s - loss: 1.1186 - accuracy: 0.5681

KeyboardInterrupt: 

In [13]:
def sliding_window(elements, window_size):
    
    a=[]
    for i in range(len(elements)):
        if len(elements)-i<window_size:
            break
        a.append(elements[i:i+window_size])
    return a

In [14]:
def position(elements, window_size):
    a=[]
    for i in range(len(elements)):
        if len(elements)-i<window_size:
            break
        a.append((i, i+window_size))
    return a

In [ ]:
pretrained_model = tf.keras.models.load_model("model_patterns_c")
for layer in pretrained_model.layers:
    layer.trainable = False
pretrained_model.predict(x_cp)

In [15]:
def evaluate(X):   
    pretrained_model = tf.keras.models.load_model("model_patterns_c")
    for layer in pretrained_model.layers:
        layer.trainable = False
    a=[] 
    b=[]   
    y=[]
    for window_size in [round(len(X)*0.1), round(len(X)*0.25), round(len(X)*0.4), round(len(X)*0.55), round(len(X)*0.7)]:
        a+=sliding_window(X, window_size)
        b+=position(X, window_size)
    
    a = pad_sequences(a, maxlen=443, dtype='float32', padding='post', value=-1)

    a = tf.convert_to_tensor(a, np.float32)  
        
    y = pretrained_model.predict(a)
          
    return y, b, a

In [16]:
y, b, a = evaluate(X_train_c[0])

2023-09-05 14:28:59.638663: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:28:59.645434: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:28:59.679363: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:29:00.052669: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.


42/42 [==============================] - 3s 51ms/step


In [17]:
def evaluation(X_train):
    Y=[]
    B=[]
    A=[]
    res=[]
    pos=[]
    for i in range(len(X_train)):
        y, b, a = evaluate(X_train[i])
        Y.append(y)
        B.append(b)
        A.append(a)
        x=[]
        z=[]
        for i in range(len(y)):
            x.append(y[i].max())
            z.append((y[i] == y[i].max())*1)
        ind = x.index(max(x))
       
        res.append(z[ind])
      
        pos.append(b[ind])
    return res, pos

In [ ]:
res, pos = evaluation(X_train_c[0:10])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
print(accuracy_score(y_train_p_c[0:10], res))
print(mean_absolute_error(y_train_dates_c[0:10], pos))

In [ ]:
funcion que evalue el patron en cada posicion
sortear a y b en funcion de el valor maximo 


In [ ]:
sin pretrained_model

Epoch 1/50
683/683 [==============================] - 4s 6ms/step - loss: 4136.4683 - mae: 42.7351 - val_loss: 3364.1133 - val_mae: 37.1307
Epoch 2/50
683/683 [==============================] - 4s 5ms/step - loss: 3231.4341 - mae: 38.6933 - val_loss: 6599.9214 - val_mae: 51.2419
Epoch 3/50
683/683 [==============================] - 4s 5ms/step - loss: 2954.7578 - mae: 36.9046 - val_loss: 7066.0229 - val_mae: 53.7811
Epoch 4/50
683/683 [==============================] - 4s 5ms/step - loss: 2796.7285 - mae: 35.6601 - val_loss: 9115.3301 - val_mae: 61.8535
Epoch 5/50

con pretrained model

Epoch 1/50
683/683 [==============================] - 29s 38ms/step - loss: 4042.2585 - mae: 42.2140 - val_loss: 3969.2393 - val_mae: 39.3837
Epoch 2/50
683/683 [==============================] - 24s 35ms/step - loss: 3080.0176 - mae: 37.9031 - val_loss: 8152.2822 - val_mae: 58.9240
Epoch 3/50
683/683 [==============================] - 25s 36ms/step - loss: 2858.9402 - mae: 36.3635 - val_loss: 7332.1855 - val_mae: 55.7956
Epoch 4/50
683/683 [==============================] - 30s 44ms/step - loss: 2779.9702 - mae: 35.6062 - val_loss: 7605.7148 - val_mae: 55.7532
Epoch 5/50
683/683 [==============================] - 25s 36ms/step - loss: 2736.2461 - mae: 35.2941 - val_loss: 8093.8960 - val_mae: 58.7626
Epoch 6/50
683/683 [==============================] - 25s 36ms/step - loss: 2690.8677 - mae: 34.9329 - val_loss: 7721.0527 - val_mae: 56.6852
Epoch 7/50
519/683 [=====================>........] - ETA: 5s - loss: 2661.7788 - mae: 34.7680

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_5)

In [30]:
pretrained_model = tf.keras.models.load_model("model_patterns_c")
for layer in pretrained_model.layers:
    layer.trainable = False
pretrained_model.layers[3]

2023-09-05 14:41:59.668130: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:41:59.674786: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:41:59.705828: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 14:42:00.362463: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.


In [18]:
def conditional_regression_model_20():
    # Input layer
    # Input layer
    input_shape = x_cp.shape[1:]
    input_layer = Input(shape=input_shape)
    pretrained_model = tf.keras.models.load_model("model_patterns_c")
    for layer in pretrained_model.layers:
        layer.trainable = False

    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    
    #maxpool_2 = AveragePooling1D(pool_size=2)(conv1d_2)
    #conv1d_3 = Conv1D(32, activation='relu', kernel_size=5)(maxpool_2)
    #maxpool_3 = AveragePooling1D(pool_size=2)(conv1d_3)
    #conv1d_4 = Conv1D(64, activation='relu', kernel_size=5)(maxpool_3)
    
    lstm_1 = (pretrained_model.layers[2])(masked_input)
    #lstm_2 = (pretrained_model.layers[3])(lstm_1)

    
    #lstm_3 = LSTM(units=16, return_sequences=True)(lstm_1)
    conv1d_1 = Conv1D(8, activation='relu', kernel_size=5)(lstm_1)
    maxpool_1 = AveragePooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(16, activation='relu', kernel_size=5)(maxpool_1)
    maxpool_2 = AveragePooling1D(pool_size=2)(conv1d_2)
    conv1d_3 = Conv1D(32, activation='relu', kernel_size=5)(maxpool_2)
    
    # Add the second LSTM layer
    #lstm_3 = LSTM(units=16, return_sequences=False)(lstm_1)

    flatten_2 = Flatten()(conv1d_3)
    dense_1d = Dense(units=32, activation='relu')(flatten_2)

    #dropout_1 = Dropout(rate=0.5)(dense_1)
    # dense_2 = Dense(units=32, activation='relu')(dense_1)
    
    # Classification output

    # Combine input and classification output for regression

    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(dense_1d)

    # Create the combined model
    model = Model(inputs=input_layer, outputs=[regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='rmsprop',
                  loss={'regression_output': 'mean_squared_error'},
                  #loss_weights={'classification_output', 'regression_output'},
                  metrics={'regression_output': 'mae'})

    return model

    


In [19]:
es = EarlyStopping(patience = 4, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_5 = conditional_regression_model_20()


history = model_5.fit(
    x=x_cp,
    y={'regression_output': y_cd},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

2023-09-05 15:28:14.289356: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 15:28:14.295809: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 15:28:14.329588: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-05 15:28:14.718394: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.


Epoch 1/50
691/691 [==============================] - 16s 21ms/step - loss: 3786.6636 - mae: 41.7285 - val_loss: 2956.5503 - val_mae: 36.7220
Epoch 2/50
691/691 [==============================] - 15s 22ms/step - loss: 2738.2454 - mae: 35.7155 - val_loss: 2577.0059 - val_mae: 34.6734
Epoch 3/50
691/691 [==============================] - 14s 21ms/step - loss: 2535.9705 - mae: 34.1570 - val_loss: 2450.1016 - val_mae: 33.8044
Epoch 4/50
691/691 [==============================] - 14s 20ms/step - loss: 2394.3599 - mae: 33.1360 - val_loss: 2327.4746 - val_mae: 32.8866
Epoch 5/50
691/691 [==============================] - 15s 22ms/step - loss: 2306.2280 - mae: 32.4594 - val_loss: 2412.3508 - val_mae: 34.4890
Epoch 6/50
691/691 [==============================] - 14s 21ms/step - loss: 2228.2332 - mae: 31.8145 - val_loss: 2914.9248 - val_mae: 36.4909
Epoch 7/50
691/691 [==============================] - 14s 21ms/step - loss: 2157.9568 - mae: 31.2205 - val_loss: 2116.7578 - val_mae: 30.5277
Epoch 

In [23]:
def conditional_regression_model_30():
    model = Sequential()
    model.add(Masking(mask_value=-1))
    model.add(Input(shape=x_cp.shape[1:]))
    model.add(LSTM(units=16, return_sequences=True))
    model.add(LSTM(units=16, return_sequences=True))
    model.add(Dense(units=1, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics='mae')

    #model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics='accuracy')
    
    return model

In [25]:
model = conditional_regression_model_30()

In [ ]:
es = EarlyStopping(patience = 4, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_5 = conditional_regression_model_20()


history = model_5.fit(
    x=x_cp,
    y={'regression_output': y_cdbool},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

In [21]:
x_cp.shape

TensorShape([27637, 436, 4])

In [22]:
def bool_dates(y_cp, maxlen):
    b=[]
    for i in range(len(y_cp)):
        a=[]
        y_cps = np.array(y_cp[i])
        a+=[0]*y_cps[0]
        a+=[1]*(y_cps[1]-y_cps[0])
        a+=[0]*(maxlen-y_cps[1])
        b.append(a)
    return b
y_cdbool = bool_dates(y_cd, 436)


When it comes to more advanced deep learning models for stock price pattern detection, you can explore several approaches that go beyond traditional LSTM or Conv1D architectures. These advanced models are designed to capture complex patterns and dependencies in financial time series data. Here are some advanced deep learning models and techniques you can consider:

1. **Transformer-Based Models:**
   - **BERT (Bidirectional Encoder Representations from Transformers):** Originally designed for natural language processing, BERT-based models can be fine-tuned for time series data to capture bidirectional dependencies in the data.

2. **Temporal Convolutional Networks (TCNs):**
   - TCNs are a more advanced version of Conv1D layers and have shown promise in modeling sequential data, including stock prices. They use causal convolutions to capture temporal dependencies efficiently.

3. **WaveNet:**
   - WaveNet is a deep generative model that uses dilated causal convolutions to model time series data. It has been used for tasks like generating financial time series data and anomaly detection.

4. **Attention-Based Models:**
   - Attention mechanisms, similar to those used in Transformers, can be applied to time series data to focus on relevant time steps and capture complex dependencies.

5. **Recurrent Neural Networks (RNN) Variants:**
   - Beyond LSTM and GRU, you can explore more advanced RNN variants like Neural ODEs (Ordinary Differential Equations) and LSTM with Attention Mechanisms for enhanced modeling of time series data.

6. **Hybrid Models:**
   - Combine deep learning models with traditional time series forecasting techniques like ARIMA (AutoRegressive Integrated Moving Average) or Prophet to capture both linear and nonlinear patterns.

7. **Generative Adversarial Networks (GANs):**
   - GANs can be used for generating synthetic financial time series data or for anomaly detection by training a GAN to distinguish between real and synthetic data.

8. **Self-Supervised Learning:**
   - Use self-supervised learning techniques like Contrastive Learning or Variational Autoencoders (VAEs) to learn meaningful representations from the data and identify patterns.

9. **Reinforcement Learning:**
   - Apply reinforcement learning techniques to develop trading algorithms that can adapt to changing market conditions based on historical data.

10. **Graph Neural Networks (GNNs):**
    - If you have access to additional data like financial news or social media sentiment, GNNs can be used to model the relationships between various financial assets and incorporate external information.

11. **Ensemble Models:**
    - Combine predictions from multiple advanced models, including deep learning and traditional models, to improve accuracy and robustness.

The choice of the most suitable advanced model depends on the specific characteristics of your data, the complexity of the patterns you aim to detect, and your computational resources. It's also essential to properly preprocess and engineer features from your financial time series data, as the quality of input features can significantly impact model performance.

Additionally, you may need to consider techniques for addressing common challenges in financial time series analysis, such as dealing with non-stationarity, handling imbalanced datasets, and implementing robust evaluation metrics.

In [43]:
import math
import numpy as np


def transform(serie):
        """Compute the Gramian Angular Field of an image"""
        # Min-Max scaling
        min_ = np.amin(serie)
        max_ = np.amax(serie)
        scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

        # Floating point inaccuracy!
        scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
        scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

        # Polar encoding
        phi = np.arccos(scaled_serie)
        # Note! The computation of r is not necessary
        r = np.linspace(0, 1, len(scaled_serie))

        # GAF Computation (every term of the matrix)
        gaf = tabulate(phi, phi, cos_sum)

        return(gaf, phi, r, scaled_serie)
    
def tabulate(x, y, f):
    """Return a table of f(x, y). Useful for the Gram-like operations."""
    return np.vectorize(f)(*np.meshgrid(x, y, sparse=True))

def cos_sum(a, b):
    """To work with tabulate."""
    return(math.cos(a+b))

def create_time_serie(size, time):
    """Generate a time serie of length size and dynamic with respect to time."""
    # Generating time-series
    support = np.arange(0, size)
    serie = np.cos(support + float(time))
    return(t, serie)


In [59]:
gaf, phi, r, scaled_serie = transform(x_cp[0][:,3])

In [152]:
X_test_preprocessed[0]

<tf.Tensor: shape=(450, 4), dtype=float32, numpy=
array([[-0.58518606, -0.00304419, -0.17458218, -0.2111447 ],
       [-0.17715593, -0.34399354,  0.05527524, -0.51998323],
       [-0.63318956, -1.1963669 , -1.3238692 , -1.7332774 ],
       ...,
       [-1.        , -1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        ]],
      dtype=float32)>

In [203]:
gafs=[]
for i in range(len(X_train_preprocessed)): 
    if i>0:
        gafs.append(gaffs)
    gaffs=[]
    for j in range(4):
        gaf, phi, r, scaled_serie = transform(X_train_preprocessed[i][:,j])
        gaffs.append(gaf)

KeyboardInterrupt: 

In [200]:

def gaf_model():
    # Define the input layer
    input_layer = Input(shape=gafs.shape[1:])
    
    # Add Convolutional layers
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    
    maxpool = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(maxpool)

    maxpool = MaxPooling2D((2, 2))(conv2)
    # Flatten the output of the last convolutional layer
    flatten = Flatten()(conv2)

    # Add Dense (fully connected) layers
    dense1 = Dense(64, activation='relu')(flatten)

    # Output layer for classification with 5 units and softmax activation
    output_layer = Dense(5, activation='softmax')(dense1)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [201]:
y_gafs = y_train_p[0:gafs.shape[0]]

In [202]:
es = EarlyStopping(patience = 5, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model = gaf_model()


history = model.fit(
    x=gafs,
    y=y_gafs,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
10/10 [==============================] - 1s 55ms/step - loss: 2.2489 - accuracy: 0.2924 - val_loss: 1.7946 - val_accuracy: 0.2763
Epoch 2/50
10/10 [==============================] - 0s 44ms/step - loss: 1.5942 - accuracy: 0.3654 - val_loss: 1.9629 - val_accuracy: 0.3026
Epoch 3/50
10/10 [==============================] - 0s 44ms/step - loss: 1.4898 - accuracy: 0.4850 - val_loss: 1.7355 - val_accuracy: 0.2237
Epoch 4/50
10/10 [==============================] - 0s 45ms/step - loss: 1.1912 - accuracy: 0.5249 - val_loss: 1.8575 - val_accuracy: 0.2237
Epoch 5/50
10/10 [==============================] - 0s 44ms/step - loss: 1.0014 - accuracy: 0.5714 - val_loss: 1.9105 - val_accuracy: 0.1842
Epoch 6/50
10/10 [==============================] - 0s 44ms/step - loss: 0.8882 - accuracy: 0.6645 - val_loss: 2.2631 - val_accuracy: 0.0921
Epoch 7/50
10/10 [==============================] - 0s 44ms/step - loss: 0.7518 - accuracy: 0.7409 - val_loss: 2.2350 - val_accuracy: 0.1579
Epoch 8/50
10